In [11]:
from dotenv import load_dotenv
import json, pprint
from pprint import pprint

import openai

In [12]:
import os, sys
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

In [13]:
# APIキーをファイルから読み込み，APIキー認証
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

### function callingの定義

ざっくりいうと，数学問題文からオブジェクト("立方体",　"垂線", など)をパラメータ付きで抽出するタスクをGPTに指示している．  
言語処理のタスクのみでGeoGebra Scriptの生成は実行していない．

In [14]:
functions = [
    {
        "name": "information_extraction_3Dshapes",
        "description": """This process extracts the part object names and parameters necessary to visualize a figure from a mathematical problem statement for a spatial graphic written in Japanese.
        Part objects include the following. This is object type name and Japanese. 
        part_object_type = [
            ("Cube", "立方体"),
            ("Rectangular", "直方体"),
            ("Segment", "交点"),
            ("Intersection", "線分"),
            ("Midpoint", "中点"),
            ("PerpendicularLine", "垂線"),
        ]

        For each part object, extract the following parameters.
        parameters = {
            "Cube": ["length of a side"],
            "Rectangular": ["length_name", "length", "depth_name", "depth", "height_name", "height"],
            "Segment": ["point_1", "point_2"],
            "Intersection": ["object_name_1", "object_name_2"],
            "Midpoint": ["point_1", "point_2"],
            "PerpendicularLine": ["point", "object_name"],
        }
        """,
        "parameters": {
            "type": "object",
            "properties": {
                "part_objects": {
                    "type": "array",
                    "description": "An array of part object types, names, and parameters needed to visualize the figure.",
                    "items": {
                        "type": "object",
                        "properties": {
                            "object_name": {
                                "type": "string",
                                "description": "A name that identifies a shape. Restrict names to symbols only, such as ABCD-EFGH. If an obvious name does not exist in the problem statement, give an appropriate name, such as 'Perpendicular_A-BC'."
                            },
                            "object_type": {
                                "type": "string",
                                "description": "The above part object type."
                            },
                            "parameters": {
                                "type": "array",
                                "description": "An array of parameters associated with the part object. Extract the parameters as shown in the parameters dictionary above.",
                                "items": {
                                    "type": "string",
                                    "description": "Parameters related to the figure. The length and the name of the object correspond to these parameters."
                                }
                            }
                        }
                    }
                }
            },
            "required": ["part_objects"]
        }
    }
]

### システムプロンプトの入力

In [15]:
system_prompt = "I enter a math problem statement in Japanese. Please, extract the part object names and parameters necessary to visualize a figure from problem statement."

### 問題文の入力

In [16]:
question_sentence = "立体ABCD-EFGHは，AB=40cm，AD=30cm，AE=50cmの直方体である．頂点Dと頂点Fを結び，頂点Bから線分DFに引いた垂線と線分DFとの交点をIとする．線分BIの長さは何cmか．"

### ChatCompletionの実行

In [17]:
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": question_sentence}
    ],
    functions=functions,
    temperature=0,
    function_call={"name": "information_extraction_3Dshapes"}
)

print(response)
print(response["choices"][0]["message"]["function_call"]["name"])
print(response["choices"][0]["message"]["function_call"]["arguments"])

{
  "id": "chatcmpl-842L2dZnNdT5XvVUHD7MVwSPKAExS",
  "object": "chat.completion",
  "created": 1695972992,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "information_extraction_3Dshapes",
          "arguments": "{\n  \"part_objects\": [\n    {\n      \"object_name\": \"ABCD-EFGH\",\n      \"object_type\": \"Rectangular\",\n      \"parameters\": [\"AB\", \"40cm\", \"AD\", \"30cm\", \"AE\", \"50cm\"]\n    },\n    {\n      \"object_name\": \"DF\",\n      \"object_type\": \"Segment\",\n      \"parameters\": [\"D\", \"F\"]\n    },\n    {\n      \"object_name\": \"BI\",\n      \"object_type\": \"PerpendicularLine\",\n      \"parameters\": [\"B\", \"DF\"]\n    },\n    {\n      \"object_name\": \"I\",\n      \"object_type\": \"Intersection\",\n      \"parameters\": [\"BI\", \"DF\"]\n    }\n  ]\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  

### GeoGebra Scriptを生成する関数群

In [18]:
def make_ggbscript_Cube(name, parameter_list):
    return "立方体\n"

def make_ggbscript_Rectangular(name, parameter_list):
    length = parameter_list[0]
    height = parameter_list[1]
    depth = parameter_list[2]

    ggbscript = f"{name[0]} = (0,0,0)\n"
    ggbscript += f"{name[1]} = ({length},0,0)\n"
    ggbscript += f"{name[2]} = ({length},{depth},0)\n"
    ggbscript += f"{name[3]} = (0,{depth},0)\n"
    ggbscript += f"{name[5]} = (0,0,{height})\n"
    ggbscript += f"{name[6]} = ({length},0,{height})\n"
    ggbscript += f"{name[7]} = ({length},{depth},{height})\n"
    ggbscript += f"{name[8]} = (0,{depth},{height})\n"
    
    ggbscript += f"{name[0]}{name[1]} = Segment({name[0]},{name[1]})\n"
    ggbscript += f"{name[1]}{name[2]} = Segment({name[1]},{name[2]})\n"
    ggbscript += f"{name[2]}{name[3]} = Segment({name[2]},{name[3]})\n"
    ggbscript += f"{name[3]}{name[0]} = Segment({name[3]},{name[0]})\n"
    ggbscript += f"{name[0]}{name[5]} = Segment({name[0]},{name[5]})\n"
    ggbscript += f"{name[1]}{name[6]} = Segment({name[1]},{name[6]})\n"
    ggbscript += f"{name[2]}{name[7]} = Segment({name[2]},{name[7]})\n"
    ggbscript += f"{name[3]}{name[8]} = Segment({name[3]},{name[8]})\n"
    ggbscript += f"{name[5]}{name[6]} = Segment({name[5]},{name[6]})\n"
    ggbscript += f"{name[6]}{name[7]} = Segment({name[6]},{name[7]})\n"
    ggbscript += f"{name[7]}{name[8]} = Segment({name[7]},{name[8]})\n"
    ggbscript += f"{name[8]}{name[5]} = Segment({name[8]},{name[5]})\n"

    return ggbscript

def make_ggbscript_Segment(name, parameter_list):
    ggbscript = f"{name} = Segment({parameter_list[0]},{parameter_list[1]})\n"
    return ggbscript

def make_ggbscript_Intersection(name, parameter_list):
    ggbscript = f"{name} = Intersect({parameter_list[0]},{parameter_list[1]})\n"
    return ggbscript

def make_ggbscript_Midpoint(name, parameter_list):
    return "中点\n"

def make_ggbscript_PerpendicularLine(name, parameter_list):
    ggbscript = f"{name} = PerpendicularLine({parameter_list[0]},{parameter_list[1]})\n"
    return ggbscript

In [19]:
def information_extraction_3Dshapes(part_object_list:list):
    make_ggbscript_functions = {
        "Cube": make_ggbscript_Cube,
        "Rectangular": make_ggbscript_Rectangular,
        "Segment": make_ggbscript_Segment,
        "Intersection": make_ggbscript_Intersection,
        "Midpoint": make_ggbscript_Midpoint,
        "PerpendicularLine": make_ggbscript_PerpendicularLine,
    }
    ggbscript_list = []
    for part_object in part_object_list:
        if part_object["object_type"] in make_ggbscript_functions.keys():
            function_to_call = make_ggbscript_functions[part_object["object_type"]]
            ggbscript = function_to_call(part_object["object_name"], part_object["parameters"])
        else:
            raise ValueError(f"{part_object['object_type']} は想定外のobject typeです．GeoGebraを生成するための関数が存在しません．")
        ggbscript_list.append(ggbscript)
    return ggbscript_list


### 実行と実行結果

In [20]:
available_functions = {
    "information_extraction_3Dshapes": information_extraction_3Dshapes,
}

response_message = response["choices"][0]["message"]
if response_message.get("function_call"):
    # 呼び出す関数の名前を取り出す, string
    function_name = response_message["function_call"]["name"]
    fuction_to_call = available_functions[function_name] # fuction_to_callは実行すべき関数
    # 関数の引数を辞書型に変換して取り出す
    function_arguments = json.loads(response_message["function_call"]["arguments"])
    # 関数を実行
    function_response = fuction_to_call(function_arguments["part_objects"])

    for ggbscript in function_response:
        print(ggbscript, end="")
    

A = (0,0,0)
B = (AB,0,0)
C = (AB,AD,0)
D = (0,AD,0)
E = (0,0,40cm)
F = (AB,0,40cm)
G = (AB,AD,40cm)
H = (0,AD,40cm)
AB = Segment(A,B)
BC = Segment(B,C)
CD = Segment(C,D)
DA = Segment(D,A)
AE = Segment(A,E)
BF = Segment(B,F)
CG = Segment(C,G)
DH = Segment(D,H)
EF = Segment(E,F)
FG = Segment(F,G)
GH = Segment(G,H)
HE = Segment(H,E)
DF = Segment(D,F)
BI = PerpendicularLine(B,DF)
I = Intersect(BI,DF)
